In [173]:
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import keras as ks
import sklearn.metrics as metrics
import pandas as pd

In [2]:
data = "C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\granulomas_final.h5ad"

In [3]:
adata = sc.read_h5ad(data)
adata.uns['log1p']["base"] = None # bug fix

print(adata.obs['sample'].value_counts())
print()
print('X matrix is sparse:', scipy.sparse.issparse(adata.X))
print('X size =', adata.X.shape)

sample
granuloma2    9392
granuloma1    9302
granuloma3    8909
Name: count, dtype: int64

X matrix is sparse: False
X size = (27603, 23693)


In [4]:
cluster_type = 'my_clust_1'

In [5]:
annotation_dict= {
    '9': 'CAP1',
    '24': 'CAP2',
    '9b': 'VEC',
    '27': 'LEC',
        
    '17': 'Ciliated',
    '15': 'Secretory',
    '22': 'AT1',
    '6': 'AT2',
    '12': 'AT2-t1',
    '19': 'AT2-t2',
        
    '14': 'AF',
    '25': 'Pericyte',
        
    '20': 'Mesothelial',
        
    '3': 'B1',
    '3b': 'B2',
        
    '0': 'Th1',
    '8': 'Tnaive',
    '11': 'Tex',
    '77': 'Treg',
        
    '11b': 'NK',
        
    '4a': 'AM',
    '4': 'M-t1',
    '10': 'M-lc',
    '7': 'M-t2',
    '7b': 'M-C1q',
    '7c': 'iMon',
        
    '23': 'pDC',
    '13': 'DC',
    '5b': 'N1',
    '5': 'N2',
}

In [6]:
adata.obs['cell_type_edit'] = [annotation_dict[clust] for clust in adata.obs[cluster_type]]

dict_list = list(annotation_dict.keys())
adata_list = list(adata.obs[cluster_type].unique())
print('Keys in dictionary not in adata:', [item for item in dict_list if item not in adata_list])
print('Keys in adata not in dictionary:', [item for item in adata_list if item not in dict_list])

Keys in dictionary not in adata: []
Keys in adata not in dictionary: []


In [7]:
replacement_dict = {
    'AT2': 0,
    'B1': 1,
    'M-t1': 2,
    'DC': 3,
    'Th1': 4,
    'M-t2': 5,
    'Secretory': 6,
    'AM': 7,
    'N1': 8,
    'M-C1q': 9,
    'AT2-t2': 10,
    'AF': 11,
    'VEC': 12,
    'CAP1': 13,
    'N2': 14,
    'AT2-t1': 15,
    'Pericyte': 16,
    'pDC': 17,
    'Ciliated': 18,
    'NK': 19,
    'AT1': 20,
    'Tnaive': 21,
    'Treg': 22,
    'M-lc': 23,
    'Mesothelial': 24,
    'Tex': 25,
    'CAP2': 26,
    'LEC': 27,
    'iMon': 28,
    'B2': 29
}

In [8]:
adata.obs['celltype'] = adata.obs['cell_type_edit'].map(replacement_dict)

In [9]:
FINAL_SEED = 99

subset, global_test = train_test_split(adata.obs.index.values, test_size=0.2, random_state=FINAL_SEED, stratify=adata.obs['celltype'].values)
print('subset shape', subset.shape)
print('global_test shape', global_test.shape)

subset shape (22082,)
global_test shape (5521,)


In [10]:
adata_subset = adata[subset].copy()
adata_global_test = adata[global_test].copy()

adata_subset.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_subset.h5ad")
adata_global_test.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_global_test.h5ad")

adata = adata_subset

In [11]:
def preprocess_node(cell_name, cell_dict, replacement_dict, seed1, seed2):

    cell_list = [c for lst in cell_dict.values() for c in lst]
    adata_subset = adata[adata.obs['cell_type_edit'].isin(cell_list)].copy()
    annotation = {}

    for cell_type, cluster_num in cell_dict.items():
        for x in cluster_num:
            annotation[x] = cell_type

    adata_subset.obs["annotation"] = adata_subset.obs['cell_type_edit'].map(annotation)
    adata_subset.obs['celltype'] = adata_subset.obs["annotation"].map(replacement_dict)
    adata_subset_hvg = adata_subset[:, adata_subset.var['highly_variable']].copy()

    X = adata_subset_hvg.X
    y = adata_subset_hvg.obs['celltype'].values
    
    train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.2, random_state=seed1, stratify=y)
    train_features, val_features, train_labels, val_labels = train_test_split(train_features, train_labels, test_size = 0.125, random_state=seed2, stratify=train_labels)
    weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)  

    train_features = np.array(train_features)
    test_features = np.array(test_features)
    val_features = np.array(val_features)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)
    val_labels = np.array(val_labels)
    weights = np.array(weights)                  

    np.save(f'../Arrays/train_features_hvg_{cell_name}.npy', train_features)
    np.save(f'../Arrays/test_features_hvg_{cell_name}.npy', test_features)
    np.save(f'../Arrays/val_features_hvg_{cell_name}.npy', val_features)
    np.save(f'../Arrays/train_labels_hvg_{cell_name}.npy', train_labels)
    np.save(f'../Arrays/test_labels_hvg_{cell_name}.npy', test_labels)
    np.save(f'../Arrays/val_labels_hvg_{cell_name}.npy', val_labels)
    np.save(f'../Arrays/weights_hvg_{cell_name}.npy', weights)

    adata_subset_hvg.write(f"C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_hvg_{cell_name}.h5ad")

In [12]:
def preprocess_leaf(cell_name, cell_list, replacement_dict, seed1, seed2):

    adata_subset = adata[adata.obs['cell_type_edit'].isin(cell_list)].copy()
    adata_subset.obs['celltype'] = adata_subset.obs['cell_type_edit'].map(replacement_dict)
    adata_subset_hvg = adata_subset[:, adata_subset.var['highly_variable']].copy()

    X = adata_subset_hvg.X
    y = adata_subset_hvg.obs['celltype'].values

    train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.2, random_state=seed1, stratify=y)
    train_features, val_features, train_labels, val_labels = train_test_split(train_features, train_labels, test_size = 0.125, random_state=seed2, stratify=train_labels)
    weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)  

    train_features = np.array(train_features)
    test_features = np.array(test_features)
    val_features = np.array(val_features)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)
    val_labels = np.array(val_labels)
    weights = np.array(weights)                  

    np.save(f'../Arrays/train_features_hvg_{cell_name}.npy', train_features)
    np.save(f'../Arrays/test_features_hvg_{cell_name}.npy', test_features)
    np.save(f'../Arrays/val_features_hvg_{cell_name}.npy', val_features)
    np.save(f'../Arrays/train_labels_hvg_{cell_name}.npy', train_labels)
    np.save(f'../Arrays/test_labels_hvg_{cell_name}.npy', test_labels)
    np.save(f'../Arrays/val_labels_hvg_{cell_name}.npy', val_labels)
    np.save(f'../Arrays/weights_hvg_{cell_name}.npy', weights)

    adata_subset_hvg.write(f"C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_hvg_{cell_name}.h5ad")

In [13]:
top_level_dict = {
    'Endothelial': ['CAP1','CAP2','VEC','AEC','LEC'],
    'Epithelial': ['Ciliated','Secretory','AT1','AT2','AT2-t1','AT2-t2'],
    'Mesenchyme': ['AF','Pericyte','SMC','Mesothelial'],
    'Immune': ['B1','B2','Th1','Tnaive','Treg','Tex','NK','AM','M-t1','M-t2','M-C1q','M-lc','iMon','DC','pDC','N1','N2']
}

endothelial_dict = {
    'Blood vessels': ['CAP1','CAP2','VEC','AEC'],
    'Lymphatic EC': ['LEC']
}

epithelial_dict = {
    'Airway epithelium': ['Ciliated','Secretory'],
    'Alveolar epithelium': ['AT1','AT2','AT2-t1','AT2-t2']
}

mesenchyme_dict = {
    'Stromal': ['AF','Pericyte','SMC'],
    'Mesothelial': ['Mesothelial']
}

immune_dict = {
    'Lymphoid': ['B1','B2','Th1','Tnaive','Treg','Tex','NK'],
    'Myeloid': ['AM','M-t1','M-t2','M-C1q','M-lc','iMon','DC','pDC','N1','N2']
}

lymphoid_dict = {
    'B lineage': ['B1','B2'],
    'T lineage': ['Th1','Tnaive','Treg','Tex'],
    'NK': ['NK']
}

myeloid_dict = {
    'mononuclear': ['AM','M-t1','M-t2','M-C1q','M-lc','iMon','DC','pDC'],
    'polymorphonuclear': ['N1','N2']
}

mononuclear_dict = {
    'Macrophage': ['AM','M-t1','M-t2','M-C1q','M-lc'],        
    'mononuclear': ['iMon','DC','pDC']
}

blood_vessels_list = ['CAP1','CAP2','VEC','AEC']
airway_epithelium_list = ['Ciliated','Secretory']
alveolar_epithelium_list = ['AT1','AT2','AT2-t1','AT2-t2']
stromal_list = ['AF','Pericyte','SMC']
b_lineage_list = ['B1','B2']
t_lineage_list = ['Th1','Tnaive','Treg','Tex']
macrophage_list = ['AM','M-t1','M-t2','M-C1q','M-lc']
mononuclear_list = ['iMon','DC','pDC']
polymorphonuclear_list = ['N1','N2']

In [14]:
replacement_dict_top_level = {
    'Endothelial': 0,
    'Epithelial': 1,
    'Mesenchyme': 2,
    'Immune': 3
}

replacement_dict_endothelial = {
    'Blood vessels': 0,
    'Lymphatic EC': 1
}

replacement_dict_epithelial = {
    'Airway epithelium': 0,
    'Alveolar epithelium': 1
}

replacement_dict_mesenchyme = {
    'Stromal': 0,
    'Mesothelial': 1
}

replacement_dict_immune = {
    'Lymphoid': 0,
    'Myeloid': 1
}

replacement_dict_lymphoid = {
    'B lineage': 0,
    'T lineage': 1,
    'NK': 2
}

replacement_dict_myeloid = {
    'mononuclear': 0,
    'polymorphonuclear': 1
}

replacement_dict_mononuclear = {
    'Macrophage': 0,
    'mononuclear': 1
}

replacement_dict_blood_vessels = {
    'VEC': 0,
    'CAP1': 1,
    'CAP2': 2,
    'AEC': 3
}

replacement_dict_airway_epithelium = {
    'Ciliated': 0,
    'Secretory': 1
}

replacement_dict_alveolar_epithelium = {
    'AT1': 0,
    'AT2': 1,
    'AT2-t1': 2,
    'AT2-t2': 3
}

replacement_dict_stromal = {
    'AF': 0,
    'Pericyte': 1,
    'SMC': 2
}

replacement_dict_b_lineage = {
    'B1': 0,
    'B2': 1
}

replacement_dict_t_lineage = {
    'Th1': 0,
    'Tnaive': 1,
    'Treg': 2,
    'Tex': 3
}

replacement_dict_macrophage = {
    'AM': 0,
    'M-t1': 1,
    'M-t2': 2,
    'M-C1q': 3,
    'M-lc': 4
}

replacement_dict_mononuclear_child = {
    'iMon': 0,
    'DC': 1,
    'pDC': 2
}

replacement_dict_polymorphonuclear = {
    'N1': 0,
    'N2': 1
}

In [15]:
preprocess_node("top_level", top_level_dict, replacement_dict_top_level, seed1=6, seed2=20)
preprocess_node("endothelial", endothelial_dict, replacement_dict_endothelial, seed1=29, seed2=96)
preprocess_node("epithelial", epithelial_dict, replacement_dict_epithelial, seed1=773, seed2=413)
preprocess_node("mesenchyme", mesenchyme_dict, replacement_dict_mesenchyme, seed1=438, seed2=210)
preprocess_node("immune", immune_dict, replacement_dict_immune, seed1=589, seed2=262)
preprocess_node("lymphoid", lymphoid_dict, replacement_dict_lymphoid, seed1=92, seed2=956)
preprocess_node("myeloid", myeloid_dict, replacement_dict_myeloid, seed1=310, seed2=341)
preprocess_node("mononuclear", mononuclear_dict, replacement_dict_mononuclear, seed1=278, seed2=999)

preprocess_leaf("blood_vessels", blood_vessels_list, replacement_dict_blood_vessels, seed1=881, seed2=114)
preprocess_leaf("airway_epithelium", airway_epithelium_list, replacement_dict_airway_epithelium, seed1=248, seed2=80)
preprocess_leaf("alveolar_epithelium", alveolar_epithelium_list, replacement_dict_alveolar_epithelium, seed1=95, seed2=102)
preprocess_leaf("stromal", stromal_list, replacement_dict_stromal, seed1=430, seed2=879)
preprocess_leaf("b_lineage", b_lineage_list, replacement_dict_b_lineage, seed1=666, seed2=36)
preprocess_leaf("t_lineage", t_lineage_list, replacement_dict_t_lineage, seed1=622, seed2=995)
preprocess_leaf("macrophage", macrophage_list, replacement_dict_macrophage, seed1=699, seed2=604)
preprocess_leaf("mononuclear_child", mononuclear_list, replacement_dict_mononuclear_child, seed1=228, seed2=462)
preprocess_leaf("polymorphonuclear", polymorphonuclear_list, replacement_dict_polymorphonuclear, seed1=365, seed2=370)

In [ ]:
name = "macrophage"
adata = sc.read_h5ad(f"C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_hvg_{name}.h5ad")

print(adata)
print()
print(adata.obs["cell_type_edit"].value_counts())
print(adata.obs["cell_type_edit"].unique())
print()
# print(adata.obs["annotation"].value_counts())
# print(adata.obs["annotation"].unique())
print()
print(adata.obs["celltype"].value_counts())
print(adata.obs["celltype"].unique())
print()

train_features = np.load(f"../Arrays/train_features_hvg_{name}.npy")
test_features = np.load(f"../Arrays/test_features_hvg_{name}.npy")
val_features = np.load(f"../Arrays/val_features_hvg_{name}.npy")
train_labels = np.load(f"../Arrays/train_labels_hvg_{name}.npy")
test_labels = np.load(f"../Arrays/test_labels_hvg_{name}.npy")
val_labels = np.load(f"../Arrays/val_labels_hvg_{name}.npy")
weights = np.load(f"../Arrays/weights_hvg_{name}.npy")

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)
print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)
print('Weights shape:', weights.shape)
print()

class_weights = dict(enumerate(weights))
print(class_weights)

In [137]:
class Node:
    def __init__(self, name, model=None, children=None):
        self.name = name
        self.model = model
        self.children = children

    def predict(self, gene):
        if (self.children == None):
            return self.name

        if (self.model == None): 
            return self.children[0].predict(gene)
        
        prediction = self.model.predict(np.array([gene]))
        max_index = np.argmax(prediction, axis=1)[0]
        return self.children[max_index].predict(gene)

In [142]:
def load_model(name):
    return ks.models.load_model(f"../Models/granulomas30_hvg_{name}_jax_v1.keras", custom_objects={'LeakyReLU': ks.layers.LeakyReLU}, compile=False)

In [45]:
def create_tree():
    CAP1 = Node("CAP1")
    CAP2 = Node("CAP2")
    VEC = Node("VEC")
    LEC = Node("LEC")
    Ciliated = Node("Ciliated")
    Secretory = Node("Secretory")
    AT1 = Node("AT1")
    AT2 = Node("AT2")
    AT2_t1 = Node("AT2-t1")
    AT2_t2 = Node("AT2-t2")
    AF = Node("AF")
    Pericyte = Node("Pericyte")
    Mesothelial = Node("Mesothelial")
    B1 = Node("B1")
    B2 = Node("B2")
    Th1 = Node("Th1")
    Tnaive = Node("Tnaive")
    Tex = Node("Tex")
    Treg = Node("Treg")
    NK = Node("NK")
    AM = Node("AM")
    M_t1 = Node("M-t1")
    M_lc = Node("M-lc")
    M_t2 = Node("M-t2")
    M_C1q = Node("M-C1q")
    iMon = Node("iMon")
    pDC = Node("pDC")
    DC = Node("DC")
    N1 = Node("N1")
    N2 = Node("N2")
    
    blood_vessels = Node(
        name = "blood_vessels",
        model = load_model("blood_vessels"), 
        children = {
            0: VEC,
            1: CAP1,
            2: CAP2
        }
    )

    lymphatic_ec = Node(
        name = "lymphatic_ec",
        model = None,
        children = {
            0: LEC
        }
    )

    airway_epithelium = Node(
        name = "airway_epithelium",
        model = load_model("airway_epithelium"), 
        children = {
            0: Ciliated,
            1: Secretory
        }
    )

    alveolar_epithelium = Node(
        name = "alveolar_epithelium",
        model = load_model("alveolar_epithelium"),
        children = {
            0: AT1,
            1: AT2,
            2: AT2_t1,
            3: AT2_t2
        }
    )

    stromal = Node(
        name = "stromal",
        model = load_model("stromal"),
        children = {
            0: AF,
            1: Pericyte,
        }
    )

    mesothelial = Node(
        name = "mesothelial",
        model = None,
        children = {
            0: Mesothelial
        }
    )

    b_lineage = Node(
        name = "b_lineage",
        model = load_model("b_lineage"),
        children = {
            0: B1,
            1: B2
        }
    )

    t_lineage = Node(
        name = "t_lineage",
        model = load_model("t_lineage"),
        children = {
            0: Th1,
            1: Tnaive,
            2: Treg,
            3: Tex
        }
    )

    nk = Node(
        name = "nk",
        model = None,
        children = {
            0: NK
        }
    )

    macrophage = Node(
        name = "macrophage",
        model = load_model("macrophage"),
        children = {
            0: AM,
            1: M_t1,
            2: M_t2,
            3: M_C1q,
            4: M_lc
        }
    )

    mononuclear_child = Node(
        name = "mononuclear_child",
        model = load_model("mononuclear_child"),
        children = {
            0: iMon,
            1: DC,
            2: pDC
        }
    )

    polymorphonuclear = Node(
        name = "polymorphonuclear",
        model = load_model("polymorphonuclear"),
        children = {
            0: N1,
            1: N2
        }
    )

    mononuclear = Node(
        name = "mononuclear",
        model = load_model("mononuclear"),
        children = {
            0: macrophage,
            1: mononuclear_child
        }
    )

    myeloid = Node(
        name = "myeloid",
        model = load_model("myeloid"),
        children = {
            0: mononuclear,
            1: polymorphonuclear
        }
    )

    lymphoid = Node(
        name = "lymphoid",
        model = load_model("lymphoid"),
        children = {
            0: b_lineage,
            1: t_lineage,
            2: nk
        }
    )

    immune = Node(
        name = "immune",
        model = load_model("immune"),
        children = {
            0: lymphoid,
            1: myeloid
        }
    )

    mesenchyme = Node(
        name = "mesenchyme",
        model = load_model("mesenchyme"),
        children = {
            0: stromal,
            1: mesothelial
        }
    )

    epithelial = Node(
        name = "epithelial",
        model = load_model("epithelial"),
        children = {
            0: airway_epithelium,
            1: alveolar_epithelium
        }
    )

    endothelial = Node(
        name = "endothelial",
        model = load_model("endothelial"),
        children = {
            0: blood_vessels,
            1: lymphatic_ec
        }
    )

    top_level = Node(
        name = "top_level",
        model = load_model("top_level"),
        children = {
            0: endothelial,
            1: epithelial,
            2: mesenchyme,
            3: immune
        }
    )

    return top_level

In [48]:
adata_global_test_hvg = adata_global_test[:, adata_global_test.var['highly_variable'] ].copy()
X = adata_global_test_hvg.X
y = adata_global_test_hvg.obs['celltype'].values

In [ ]:
# took 20 minutes

root = create_tree()

max_indices = []
for gene in X:
    predicted_cell_type = root.predict(gene)
    max_indices.append(replacement_dict[predicted_cell_type])

In [ ]:
max_indices = np.array(max_indices)
np.save('../Arrays/max_indices.npy', max_indices)

In [217]:
inverse_dict = {i: j for j, i in replacement_dict.items()}
true_cells = [inverse_dict[i] for i in y]
predicted_cells = [inverse_dict[i] for i in max_indices]

In [ ]:
# hierarchical_classification.csv

overall = {
    'accuracy': metrics.accuracy_score(y, max_indices),
    'balanced_accuracy': metrics.balanced_accuracy_score(y, max_indices),
    'precision': metrics.precision_score(y, max_indices, average='weighted', zero_division=0),
    'recall': metrics.recall_score(y, max_indices, average='weighted', zero_division=0),
    'f1_score' : metrics.f1_score(y, max_indices, average='weighted', zero_division=0)
}

per_class_accuracy = {
    inverse_dict[i]: (max_indices[y == i] == i).mean()
    for i in np.unique(y)
}

class_report = metrics.classification_report(y, max_indices, target_names=[inverse_dict[i] for i in range(len(inverse_dict))], digits=2, zero_division=0)

df = pd.DataFrame(X, columns=adata_global_test_hvg.var.index.to_list())
df.insert(0, 'predicted_cell', predicted_cells)
df.insert(0, 'true_cell', true_cells)

csv_path = "Results/hierarchical_classification.csv"

with open(csv_path, 'w', newline='') as f:
    
    f.write("overall_metrics:\n")
    for k, v in overall.items(): f.write(f"{k}: {v}\n")
    
    f.write("\nper_cell_accuracy:\n")
    for k, v in per_class_accuracy.items(): f.write(f"{k}: {v}\n")

    f.write("\nclassification_report:\n")
    for line in class_report.splitlines(): f.write(f"{line}\n")
    f.write("\n")  

    df.to_csv(f, index=False)

print(f"csv written to {csv_path}")

csv written to hierarchical_classification.csv


In [230]:
model = ks.models.load_model(f"../Models/granulomas30_hvg_subset_jax_v1.keras", custom_objects={'LeakyReLU': ks.layers.LeakyReLU}, compile=False)

In [233]:
prediction = model.predict(X)
max_indices = np.argmax(prediction, axis=1)

173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [235]:
inverse_dict = {i: j for j, i in replacement_dict.items()}
true_cells = [inverse_dict[i] for i in y]
predicted_cells = [inverse_dict[i] for i in max_indices]

In [ ]:
# flat_classification.csv

overall = {
    'accuracy': metrics.accuracy_score(y, max_indices),
    'balanced_accuracy': metrics.balanced_accuracy_score(y, max_indices),
    'precision': metrics.precision_score(y, max_indices, average='weighted', zero_division=0),
    'recall': metrics.recall_score(y, max_indices, average='weighted', zero_division=0),
    'f1_score' : metrics.f1_score(y, max_indices, average='weighted', zero_division=0)
}

per_class_accuracy = {
    inverse_dict[i]: (max_indices[y == i] == i).mean()
    for i in np.unique(y)
}

class_report = metrics.classification_report(y, max_indices, target_names=[inverse_dict[i] for i in range(len(inverse_dict))], digits=2, zero_division=0)

df = pd.DataFrame(X, columns=adata_global_test_hvg.var.index.to_list())
df.insert(0, 'predicted_cell', predicted_cells)
df.insert(0, 'true_cell', true_cells)

csv_path = "Results/flat_classification.csv"

with open(csv_path, 'w', newline='') as f:
    
    f.write("overall_metrics:\n")
    for k, v in overall.items(): f.write(f"{k}: {v}\n")
    
    f.write("\nper_cell_accuracy:\n")
    for k, v in per_class_accuracy.items(): f.write(f"{k}: {v}\n")

    f.write("\nclassification_report:\n")
    for line in class_report.splitlines(): f.write(f"{line}\n")
    f.write("\n")  

    df.to_csv(f, index=False)

print(f"csv written to {csv_path}")

csv written to flat_classification.csv
